In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'italy'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

22:57:13 - cmdstanpy - INFO - Chain [1] start processing
22:57:13 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [

In [ ]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [ ]:
df_predicted, rmse = test_model(df_test, model)

In [ ]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-01-01,3.652731,3.263218,4.089276,3.652731,3.652731,0.026730,0.026730,0.026730,0.026730,0.026730,0.026730,0.0,0.0,0.0,3.679461
1,2022-02-01,3.629446,3.254580,4.051843,3.629446,3.629446,0.025427,0.025427,0.025427,0.025427,0.025427,0.025427,0.0,0.0,0.0,3.654873
2,2022-03-01,3.608414,3.199486,4.008016,3.608414,3.608414,0.024051,0.024051,0.024051,0.024051,0.024051,0.024051,0.0,0.0,0.0,3.632465
3,2022-04-01,3.585129,3.180919,4.028247,3.585129,3.585129,0.035452,0.035452,0.035452,0.035452,0.035452,0.035452,0.0,0.0,0.0,3.620580
4,2022-05-01,3.562594,3.185803,3.996914,3.556642,3.566590,0.020527,0.020527,0.020527,0.020527,0.020527,0.020527,0.0,0.0,0.0,3.583121
5,2022-06-01,3.539309,3.142857,4.013218,3.513915,3.557600,0.020912,0.020912,0.020912,0.020912,0.020912,0.020912,0.0,0.0,0.0,3.560221
6,2022-07-01,3.516774,3.124219,3.960273,3.456612,3.556189,0.027068,0.027068,0.027068,0.027068,0.027068,0.027068,0.0,0.0,0.0,3.543842
7,2022-08-01,3.493489,3.040804,3.949740,3.395127,3.571228,0.024224,0.024224,0.024224,0.024224,0.024224,0.024224,0.0,0.0,0.0,3.517713
8,2022-09-01,3.470204,3.037698,3.990514,3.317821,3.587575,0.036703,0.036703,0.036703,0.036703,0.036703,0.036703,0.0,0.0,0.0,3.506907
9,2022-10-01,3.447669,2.947724,3.957413,3.251396,3.605282,0.013461,0.013461,0.013461,0.013461,0.013461,0.013461,0.0,0.0,0.0,3.461131


In [ ]:
rmse

0.1765160742008048

In [ ]:
with open(f'{country}_prophet_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [ ]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [ ]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [ ]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,3.334247,2.973279,3.812343,3.334247,3.334247,0.050972,0.050972,0.050972,0.050972,0.050972,0.050972,0.0,0.0,0.0,3.385219
1,2023-04-01,3.310961,2.961504,3.752974,3.310961,3.310961,0.044942,0.044942,0.044942,0.044942,0.044942,0.044942,0.0,0.0,0.0,3.355903
